<a href="https://colab.research.google.com/github/luisArthurRodriguesDaSilva/estudo-data-science-eleicoes/blob/master/apura%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
def getDeputadoEstadualUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0007-e000546-r.json'

def getPresidentUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/{uf}/{uf}-c0001-e000544-r.json'

def getGovernadorUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0003-e000546-r.json'

def getDeputadoFederalUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0006-e000546-r.json'

def getSenadorUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0005-e000546-r.json'

In [67]:
import pandas as pd
import numpy as np
import requests
import pprint
from matplotlib import pyplot as plt
import seaborn as sns

#Fazendo a requisição na URL:
def trabalhoDoJuan(url,uf): #taked on 
  re=requests.get(url).json()
  candidatos = re['cand'][:10]

  df=pd.DataFrame(candidatos)
  print(df)
  df=df[['nm','n','dvt','vap','pvap']]
  df.rename(columns={'nm':'Nome do Candidato',
                  'n':'Numero do Candidato',
                  'dvt':'Validade',
                  'vap':'Votos',
                  'pvap':'(%) Votos'}, inplace=True)

  df['Horario Atualização']=re['hg']
  df['(%) Votos']=df['(%) Votos'].str.replace(',','.')
  df['(%) Votos']=df['(%) Votos'].astype(float)
  df['Votos']=df['Votos'].astype(int)
  df['Nome do Candidato']=df['Nome do Candidato'].str.replace('D&apos;',"D'")
  df=df.sort_values(by='Votos', ascending=False)

  fig, axs = plt.subplots(ncols=1, figsize=(13,6))

  ax1 = sns.barplot(
      x="Nome do Candidato",
      y='(%) Votos',
      data=df[:10],
      palette="ch:.25",
      ax=axs)

  ax1.set(xlabel="Candidatos(as)",
          ylabel="(%) Votos",
          title=f"Apuração de resultados Eleições 2022 {uf} - Presidente - (5 mais Votados)",
          ylim=(0,70))


  # Adicionando rotulo com valores no gráfico ax1:
  for p in ax1.patches:
              ax1.annotate("%.2f" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
                  ha='center', va='center', fontsize=13, color='black', xytext=(2, 5),
                  textcoords='offset points')

  plt.show()

  print (f'Sessões Totais em {uf}: {re["s"]}')
  print (f'Sessões Apuradas em {uf}: {re["st"]}')
  pct=int(re['st'])/int(re['s'])*100
  print (f'Percentual de sessões apuradas em {uf}: {round(pct,4)}%')

In [51]:
BRAZIL_UFs = ["AC","AL","AM","AP","BA","CE","DF","ES","GO","MA","MT","MS","MG","PA","PB","PR","PE","PI","RJ","RN","RO","RS","RR","SC","SE","SP","TO"]
brasil_ufs = ['ac','al','am','ap','ba','ce','df','es','go','ma','mt','ms','mg','pa','pb','pr','pe','pi','rj','rn','ro','rs','rr','sc','se','sp','to']

In [52]:
def takeData(url):
  re=requests.get(url).json()
  candidatos = re['cand']

  df=pd.DataFrame(candidatos)
  df=df[['nm','n','dvt','vap','pvap']]
  df.rename(columns={'nm':'Nome do Candidato',
                  'n':'Numero do Candidato',
                  'dvt':'Validade',
                  'vap':'Votos',
                  'pvap':'(%) Votos'}, inplace=True)
  return(df)

In [ ]:
presidentsUrls = list(map(lambda uf : getPresidentUrl(uf) ,brasil_ufs))
presidents_data = list(map(lambda x: takeData(x),presidentsUrls))

In [ ]:
PRESIDENTS_DATA = {}
for i, curr in enumerate(presidents_data):
  PRESIDENTS_DATA[brasil_ufs[i]] = curr.sort_values(by='Votos', ascending=True)

In [ ]:
PRESIDENTS_DATA['rj']

In [68]:
test = getSenadorUrl('rj')
takeData(test)

,Nome do Candidato,Numero do Candidato,Validade,Votos,(%) Votos
0,HIRAN ROEDEL,211,Válido,5120,"0,06"
1,CABO DACIOLO,120,Válido,285037,"3,49"
2,SUED HAIDAR,355,Válido,11933,"0,15"
3,CLARISSA,444,Válido,1145413,"14,02"
4,ANDRE CECILIANO,133,Válido,986676,"12,08"
5,BÁRBARA SINEDINO,160,Válido,18222,"0,22"
6,ITAGIBA,700,Válido,18224,"0,22"
7,RAUL,800,Válido,7299,"0,09"
8,DANIEL SILVEIRA,142,Anulado sub judice,1566352,"19,18"
9,PROF. HELVIO COSTA,277,Anulado sub judice,7036,"0,09"
